In [1]:
import numpy as np
import os
import urllib
import matplotlib.pyplot as plt

from random import shuffle

%matplotlib inline

We're going to use Pride and Pejustice as corpus

In [2]:
# Download the dataset if it's not already there
if not os.path.isfile('pride-and-prejustice.txt'):
    urllib.request.urlretrieve("https://www.gutenberg.org/files/1342/1342-0.txt", filename="pride-and-prejustice.txt")

In [3]:
txt = open("pride-and-prejustice.txt", "r", encoding="utf8").read()

In [4]:
print(txt[1000:1200])

ful property
of some one or other of their daughters.

“My dear Mr. Bennet,” said his lady to him one day, “have you heard that
Netherfield Park is let at last?”

Mr. Bennet replied that he had not.




Convert text into list of wors

In [5]:
import re
words = re.split('[^A-Za-z]+', txt.lower())
words = list(filter(None, words)) # Remove empty strings
sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', txt.lower())
sentences = filter(None, sentences) # Remove empty string
mapfn = lambda item: re.sub(r"[^a-z0-9]+", " ", item.lower())
sentences = list(map(mapfn, sentences))

# Generating N-Grams

## 1-Gram Model

In [6]:
# Create set of all unique words, this throws away any information about frequency however
gram1 = set(words)

print(len(gram1))

# Instead of printing all the elements in the set, create an iterator and print 20 elements only
gram1_iter = iter(gram1)
print([next(gram1_iter) for _ in range(20)])

6530
['takes', 'cruel', 'observe', 'mixed', 'kitty', 'compromised', 'sweetness', 'dishes', 'inferiority', 'importing', 'patroness', 'vexations', 'laconic', 'produced', 'ensuring', 'originator', 'eager', 'redistributing', 'openly', 'method']


In [7]:
# See the last 10 pairs
for i in range(len(words)-10, len(words)-1):
    print(words[i], words[i+1])

subscribe to
to our
our email
email newsletter
newsletter to
to hear
hear about
about new
new ebooks


## 2-Gram Model

In [8]:
word_pairs = [(words[i], words[i+1]) for i in range(len(words)-1)]
print(len(word_pairs))

gram2 = set(word_pairs)
print(len(gram2))

# Print 20 elements from gram2
gram2_iter = iter(gram2)
print([next(gram2_iter) for _ in range(20)])

125900
55640
[('had', 'taken'), ('contradicted', 'your'), ('and', 'arranged'), ('whether', 'all'), ('painful', 'confusion'), ('not', 'sink'), ('almost', 'promised'), ('spirits', 'as'), ('violates', 'the'), ('they', 'looked'), ('bingley', 'but'), ('exaggeration', 'the'), ('could', 'encourage'), ('but', 'vanity'), ('elizabeth', 'smiling'), ('him', 'supercilious'), ('almost', 'have'), ('which', 'lydia'), ('sensible', 'a'), ('hour', 'by')]


# Frequency 

## 1-gram

In [9]:
gram1_dict = dict()

# Populate 1-gram dictionary
for word in words:
    if word in gram1_dict:
        gram1_dict[word] += 1
    else:
        gram1_dict[word] = 1 # Start a new entry with 1 count since saw it for the first time.

# Turn into a list of (word, count) sorted by count from most to least
gram1 = sorted(gram1_dict.items(), key=lambda i: -i[1])

# Print top 20 most frequent words
print(gram1[:20])

def gram1_count(word):
    if word in gram1_dict:
        return gram1_dict[word]
    return 0

print(gram1_count('the'))


[('the', 4507), ('to', 4243), ('of', 3730), ('and', 3658), ('her', 2225), ('i', 2070), ('a', 2012), ('in', 1937), ('was', 1847), ('she', 1710), ('that', 1594), ('it', 1550), ('not', 1450), ('you', 1428), ('he', 1339), ('his', 1271), ('be', 1260), ('as', 1192), ('had', 1177), ('with', 1100)]
4507


## 2-gram

In [10]:
gram2 = dict()

# Populate 2-gram dictionary
for i in range(len(words)-1):
    key = (words[i], words[i+1])
    if key in gram2:
        gram2[key] += 1
    else:
        gram2[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram2 = sorted(gram2.items(), key=lambda i: -i[1])

# Print top 20 most frequent words
print(gram2[:20])

gram2_count_cache = {}

def gram2_count(word1, word2):
    if (word1, word2) not in gram2_count_cache:
        gram2_count_cache[(word1, word2)] = 0
    else:
        return gram2_count_cache[(word1, word2)]
    
    gram2_count_cache[(word1, word2)] = 0
    for ((w1, w2), count) in gram2:
        if w2 == word2 and w1 == word1:
            gram2_count_cache[(word1, word2)] += count
    return gram2_count_cache[(word1, word2)]  

print(gram2_count('of', 'the'))


[(('of', 'the'), 491), (('to', 'be'), 445), (('in', 'the'), 397), (('i', 'am'), 303), (('mr', 'darcy'), 273), (('to', 'the'), 268), (('of', 'her'), 261), (('it', 'was'), 251), (('of', 'his'), 235), (('she', 'was'), 212), (('she', 'had'), 205), (('had', 'been'), 200), (('it', 'is'), 194), (('i', 'have'), 188), (('to', 'her'), 179), (('that', 'he'), 177), (('could', 'not'), 167), (('he', 'had'), 166), (('and', 'the'), 165), (('for', 'the'), 163)]
491


## 3-gram

In [11]:
word_triplets = [(words[i], words[i+1], words[i+2]) for i in range(len(words)-2)]

gram3 = set(word_triplets)
# Print 20 elements from gram2
gram3_iter = iter(gram3)
print([next(gram3_iter) for _ in range(20)])

gram3 = dict()

# Populate 2-gram dictionary
for i in range(len(words)-2):
    key = (words[i], words[i+1], words[i+2])
    if key in gram3:
        gram3[key] += 1
    else:
        gram3[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram3 = sorted(gram3.items(), key=lambda i: -i[1])

# Print top 20 most frequent words
print(gram3[:20])

def gram3_count(word1, word2, word3):
    wordcount = 0
    for ((w1, w2, w3), count) in gram3:
        if w2 == word2 and w1 == word1 and w3 == word3:
            return count
    return 0 

gram3_count_cache = {}
def gram3_first_2_count(word1, word2):
    if (word1, word2) not in gram3_count_cache:
        gram3_count_cache[(word1, word2)] = 0
    else: 
        return gram3_count_cache[(word1, word2)]
    
    for ((w1, w2, w3), count) in gram3:
        if w1 == word1 and w2 == word2:
            gram3_count_cache[(word1, word2)] += count
    
    return gram3_count_cache[(word1, word2)] 

print(gram3_first_2_count('in', 'the'))


[('being', 'in', 'love'), ('she', 'was', 'selected'), ('bestowed', 'my', 'father'), ('person', 'well', 'placed'), ('that', 'related', 'to'), ('her', 'own', 'vanity'), ('composure', 'he', 'said'), ('simpleton', 'could', 'she'), ('your', 'favourite', 's'), ('second', 'proposal', 'to'), ('a', 'person', 'may'), ('look', 'or', 'the'), ('most', 'unaffectedly', 'modest'), ('arise', 'from', 'the'), ('but', 'our', 'own'), ('turned', 'out', 'very'), ('debt', 'mr', 'bingley'), ('delight', 'and', 'repeat'), ('looking', 'out', 'of'), ('or', 'any', 'project')]
[(('i', 'do', 'not'), 62), (('i', 'am', 'sure'), 62), (('project', 'gutenberg', 'tm'), 57), (('as', 'soon', 'as'), 55), (('she', 'could', 'not'), 50), (('that', 'he', 'had'), 37), (('it', 'would', 'be'), 34), (('in', 'the', 'world'), 34), (('i', 'am', 'not'), 32), (('the', 'project', 'gutenberg'), 31), (('i', 'dare', 'say'), 31), (('it', 'was', 'not'), 30), (('could', 'not', 'be'), 30), (('that', 'he', 'was'), 29), (('mr', 'darcy', 's'), 29), 

# Next-word prediction

Take a random word from a corpus and try to build next word as a prediction.

In [12]:
start_word = words[int(len(words)/4)]
print(start_word)

delighted


Algorithm:

1. Pick a random word
2. Find most frequent n-gram with this word as a first one
3. Pick 2nd word from that n-gram
4. repeat

In [13]:
def get2GramSentence(word, n = 50):
    for i in range(n):
        print(word, end=' ')
        # Find Next word
        word = next((element[0][1] for element in gram2 if element[0][0] == word), None)
        if not word:
            break

word = start_word
print("Start word: %s" % word)

print("2-gram sentence: \"",)
get2GramSentence(word, 20)
print("\"")

Start word: delighted
2-gram sentence: "
delighted with the whole party were to be so much as to be so much as to be so much "


## Weighted choices 

In [14]:
import random
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w > r:
         return c
      upto += w
    
def get2GramSentenceRandom(word, n = 50):
    for i in range(n):
        print(word, end=' ')
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram2 if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]

In [15]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word: %s" % word)

    print("2-gram sentence: \"", end=" ")
    get2GramSentenceRandom(word, 50)
    print("\"")

Start word: and
2-gram sentence: " and the library as i might say for the happy to be angry that the pause awkward as his sister s performance mixing with an ante chamber to your situation the wild that miss lucas and made her dirty weather should be out miss de bourgh are not a partner "
Start word: he
2-gram sentence: " he must cease to the library he to satisfy us if she to theirs i understand it and mrs collins and soon as sensibly and she could be supposed might yet had known this could would certainly had arisen repeated discussion of her father then gathered round and dull for "
Start word: she
2-gram sentence: " she likes to go to be inclined to prove that abominable mr collins at her room in essentials oh my fair opportunity of the bad as they must have delivered his behaviour was out said lady lucas whom she was never heard that this to make the instrument she could "
Start word: when
2-gram sentence: " when all were not be sorry replied she stared at civility i should come t

# Neural N-Gram Model

1. X: Build representation of words as contactenated sequence (current  word, K previous, and count n-grams ending on current word)
2. Y: log P(word|history)
2. No softmax

fnng(w[i]..)=log P(w1|w[i-1..,i-k], c[i])
c[u]=Count(w[i-1])

In [16]:
import tensorflow
import keras
from keras import backend as K

num_cores = 16

CPU = True
GPU = False

if GPU:
    num_GPU = 1
    num_CPU = 1
if CPU:
    num_CPU = 1
    num_GPU = 0

config = tensorflow.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tensorflow.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


## Build input

In [17]:
# word embeddings
from gensim.models import Word2Vec

embedding_size = 128

v2vec = Word2Vec(sentences, size=embedding_size)

C:\Users\surov\Miniconda3\envs\tf\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
import tqdm
import os.path

def get_embedding(word):
    if word not in v2vec.wv:
        return np.zeros(embedding_size)
    return v2vec.wv[word]

def prepare_nn_inputs():
    pbar = tqdm.tqdm(total=len(gram3))
    
    W = []
    C = []
    Y = []

    for (word1, word2, word3), count in gram3:
        pbar.update()

        w = np.zeros((L, embedding_size))
        w[0] = get_embedding(word1)
        w[1] = get_embedding(word2)
        W.append(w)

        c = np.zeros((3, 2))
        c[0, 0] = 0
        c[0, 1] = gram1_count(word1)
        c[1, 0] = gram2_count(word1, word2)
        c[1, 1] = gram1_count(word2)  
        c[2, 0] = gram2_count(word2, word3)
        c[2, 1] = gram1_count(word3)

        C.append(c)

        y = np.log(count / gram3_first_2_count(word1, word2))
        Y.append(y)
    
    W = np.array(W)
    C = np.array(C)
    Y = np.array(Y)

    np.save("W.npy", W)
    np.save("C.npy", C)
    np.save("Y.npy", Y)

    pbar.close()
    return (W, C, Y)

N = 2 # n-gram output depth
L = 2 # n-gram input statistics depth

W = []
C = []
Y = []

if not os.path.isfile("W.npy"):
    (_W, _C, _Y) = prepare_nn_inputs()
    W = _W
    C = _C
    Y = _Y
else:
    W = np.load("W.npy")
    C = np.load("C.npy")
    Y = np.load("Y.npy")

print("W: %s\tC: %s" % (W.shape, C.shape))
       

W: (105491, 2, 128)	C: (105491, 3, 2)


## Construct Model

### Keras

In [90]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate, Activation, Reshape, Flatten


def build_nn_gram_model():
    input1 = Input(shape=(L, embedding_size), name='embeddings_input')
    reshape1 = Reshape((-1, L*embedding_size))(input1)
    flatten1 = Flatten()(reshape1)
    x1 = Dense(128, activation='relu')(flatten1)
    
    input2 = Input(shape=((L+1), N), name='counts_input')
    reshape2 = Reshape((-1, (L+1)*N))(input2)
    flatten2 = Flatten()(reshape2)
    x2 = Dense(32, activation='relu')(flatten2)
    
    x3 = concatenate([x1, x2])
    y = Dense(1, activation='relu', name='logp')(x1)

    model = Model(inputs=[input1], outputs=y)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# this one is OK
def build_simple_sequential_model():
    model = Sequential()
    model.add(Reshape((-1, L*embedding_size), input_shape=(L, embedding_size), name='embeddings'))
    model.add(Flatten()) 
    model.add(Dense(1, activation='relu', name='logp'))

    model.compile(loss='mean_squared_error' ,optimizer='adam')
    
    return model

def build_simple_functional_model():
    input1 = Input(shape=(L, embedding_size), name='embeddings_input')
    reshape1 = Reshape((-1, L*embedding_size))(input1)
    flatten1 = Flatten()(reshape1)
    y = Dense(1, activation='relu', name='logp')(flatten1)

    model = Model(inputs=input1, outputs=y)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


In [91]:
simple_model_sequiential = build_simple_sequential_model()
print("\n\nSequential Model:")
print(simple_model_sequiential.summary())



Sequential Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeddings (Reshape)         (None, 1, 256)            0         
_________________________________________________________________
flatten_49 (Flatten)         (None, 256)               0         
_________________________________________________________________
logp (Dense)                 (None, 1)                 257       
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________
None


In [87]:
simple_model_functional = build_simple_functional_model()
print("\n\nFuncational Model:")
print(simple_model_sequiential.summary())



Funcational Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeddings (Reshape)         (None, 1, 256)            0         
_________________________________________________________________
flatten_45 (Flatten)         (None, 256)               0         
_________________________________________________________________
logp (Dense)                 (None, 1)                 257       
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________
None


In [88]:
model = build_nn_gram_model()
print("\n\nFuncational Model:")
print(model.summary())



Funcational Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeddings (InputLayer)      (None, 2, 128)            0         
_________________________________________________________________
reshape_40 (Reshape)         (None, 1, 256)            0         
_________________________________________________________________
flatten_47 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               32896     
_________________________________________________________________
logp (Dense)                 (None, 1)                 129       
Total params: 33,025
Trainable params: 33,025
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
params = {
    'embeddings_input': W,
    'counts_input': C,
}


In [84]:
simple_model_functional.fit(params, Y, epochs=1, validation_split=0.1, batch_size=128, verbose=1)

Train on 94941 samples, validate on 10550 samples
Epoch 1/1
94941/94941 [==============================] - 1s 8us/step - loss: 3.8451 - val_loss: 3.5125


In [85]:
simple_model_sequiential.fit(params, Y, epochs=1, validation_split=0.1, batch_size=128, verbose=1)

Train on 94941 samples, validate on 10550 samples
Epoch 1/1
94941/94941 [==============================] - 1s 15us/step - loss: 3.8462 - val_loss: 3.5125


In [89]:
model.fit(params, Y, epochs=1, validation_split=0.1, batch_size=128, verbose=1)

ValueError: No data provided for "embeddings". Need data for each key in: ['embeddings']